# Objective: Scrape everything on the page
 - creating breif of each legislator that contains sentator and rep info for each distict (for a 3rd party)
     - legislative name/contact info
     - picture
     - capital office location
     - district office location 
     - JJ usually prints a page ith senator info on front, and rep on back
        

In [1]:
# import dependacies 

from splinter import Browser
from bs4 import BeautifulSoup
import requests
import re

In [2]:
# set up the chrome driever
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [4]:
# IL Genearl Assembly homepage
url = 'http://www.ilga.gov'

In [5]:
# retrieve the webpage, create BeautifulSoup object to parse the data
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
# print(soup.prettify())

the tabel is in the body of the page
    this tabel is in the td of another table
  
    this div is the td of a table
    <div>
    <table>
    <tbody>
    <tr>
    <td>
    <a class="homemenu" href="/senate/" title="View the biography, sponsored bills and committees for each Senate member">
                Members

In [6]:
# Locate link to all Senators and go to the page 
links = soup.find_all(href=re.compile("/senate/"), text='Members')

for link in links:
    reference = link.get('href')
#     browser.click_link_by_partial_text('Members')
    senate_url = url+reference
print(senate_url)
browser.visit(senate_url)
html = browser.html
senate_soup = BeautifulSoup(html, 'html.parser')
# print(senate_soup.prettify())

http://www.ilga.gov/senate/


In [7]:
# for link in senate_soup.find_all('a'):
#     print(link.get('href'))

senator_links = senate_soup.find_all('a', {'class':'notranslate'})
# senate_soup.find_all('a')

# create empty list to hold all member urls
senator_link_list = []

# loop though senate page and add each member link to the senator_link_list
for link in senator_links:
    reference = link.get('href')
#     browser.click_link_by_partial_text('Members')
    senator_url = url+reference
    senator_link_list.append(senator_url)
    
# print(senator_link_list[0])

### Work with the link to the 1st senator's page, create series of functions to obtain the information on thier page. 

In [8]:
# visit the first senator page

browser.visit(senator_link_list[0])
html = browser.html
senator_soup = BeautifulSoup(html, 'html.parser')
# print(senator_soup.prettify)

<bound method Tag.prettify of <html lang="en" style="height: 100%;" xmlns="http://www.w3.org/1999/xhtml"><!-- Trigger/Open The Modal --><head><link charset="UTF-8" href="https://translate.googleapis.com/translate_static/css/translateelement.css" rel="stylesheet" type="text/css"/><script charset="UTF-8" src="https://translate.googleapis.com/translate_static/js/element/main.js" type="text/javascript"></script><script charset="UTF-8" src="https://translate.googleapis.com/element/TE_20190916_00/e/js/element/element_main.js" type="text/javascript"></script></head><body alink="#9933FF" bgcolor="#FFFFFF" leftmargin="0" link="#3366FF" style="position: relative; min-height: 100%; top: 0px;" text="#000000" topmargin="0" vlink="#663366"><div style="position: fixed; z-index: 999; top: 5; left: 600; background-color: navy; display: block">
<button id="myBtn" style="color: white; background-color: navy; display: block">Translate Website</button></div>
<!-- The Modal -->
<div class="modal" id="myModa

In [23]:
# Obtain the senator's and district number
senator_name = senator_soup.find('span', {'class':'notranslate'}).text
senator_district = senator_soup.find('span', {'class':'heading2 notranslate'}).text

# print(senator_name)
# print(senator_district)

In [58]:
# Obtain the link to the image of the senator 

# Obtain the parital image link from the senator's webpage
senator_img_partial = senator_soup.find('table', {'class' : 'notranslate'}).img['src']
# senator_img_partial


# obtain the full image link by combining the orignal il.gov url with the partial senator image link
senator_img = url + senator_img_partial
# senator_img

In [64]:
# Get the leadership name
senator_position_all = senator_soup.find_all('span', {'class':'heading2'})
senator_position[3].text

'Assistant Republican Leader'

In [67]:
# Most of the sentor info are in tables, so can gather the address and committee assignments from td in the "member" class
senator_info_list = senator_soup.find_all('td', {'class': 'member'})
senator_info_list

[<td class="member" valign="top" width="100%"><br/><b>Springfield Office:</b></td>,
 <td class="member" width="100%">Senator 36th District</td>,
 <td class="member" width="100%">M103D  Capitol Building</td>,
 <td class="member" width="100%">Springfield, IL   62706    </td>,
 <td class="member" width="100%">(217) 782-5957</td>,
 <td class="member" width="100%"> </td>,
 <td class="member" width="100%"></td>,
 <td class="member" width="100%"><b>District Office:</b></td>,
 <td class="member" width="100%">1523 47th Ave.</td>,
 <td class="member" width="100%">Suite 2</td>,
 <td class="member" width="100%">Moline, IL  61265    </td>,
 <td class="member" width="100%">(309) 736-7084</td>,
 <td class="member"><b>Years served: </b> <br/><br/><b>Committee assignments:</b> Committee of the Whole; Commerce and Economic Development; Energy and Public Utilities; Licensed Activities (Minority Spokesperson); Transportation; Veterans Affairs; Subcommittee on Capital (TR); Oversight Medicaid Mang. Care, S

In [71]:
# Springfield office information

senator_springfield_address_line1 = senator_info_list[1].text
# senator_springfield_address_line1
senator_springfield_address_line2 = senator_info_list[2].text
senator_springfield_address_line3 = senator_info_list[3].text

senator_springfield_phone_number = senator_info_list[4].text


In [73]:
#  District office information

senator_district_address_line1 = senator_info_list[8].text

senator_district_address_line2 = senator_info_list[9].text
senator_district_address_line3 = senator_info_list[10].text

senator_district_phone_number = senator_info_list[11].text

# senator_district_address_line1

'1523 47th Ave.'

In [77]:
# Senator info: The # of years served, committee assignments, and biographhy are all under the same child tag

senator_bio_info  = senator_info_list[12].text
senator_bio_info

'Years served:  Committee assignments: Committee of the Whole; Commerce and Economic Development; Energy and Public Utilities; Licensed Activities (Minority Spokesperson); Transportation; Veterans Affairs; Subcommittee on Capital (TR); Oversight Medicaid Mang. Care, Spec; Opioid Crisis Abatement Spec. Com..Biography: '

In [87]:
ass_reps_tag = senator_soup.find_all('a', {'class': 'notranslate'})
ass_reps_tag

[<a class="notranslate" href="/house/Rep.asp?MemberID=2688">Tony McCombie</a>,
 <a class="notranslate" href="/house/Rep.asp?MemberID=2685">Michael Halpin</a>]

In [92]:
current_senator_assoc_reps = []
for tag in ass_reps_tag:
    rep = tag.text
    current_senator_assoc_reps.append(rep)
#     print(rep)
# current_senator_assoc_reps

## Create a function that will go to each senator's page and scrape thier information.